# Exercises: Python Type System & Validation

Practice what you've learned by completing these exercises.

---

In [ ]:
# Import required modules
from pydantic import BaseModel, Field, field_validator, model_validator
from typing import List, Dict, Optional, Union, Literal, Any
from enum import Enum

## Exercise 1: Database Connection Config

**Goal**: Create a `DatabaseConfig` model with proper types and constraints.

**Requirements**:
- `host`: str (required)
- `port`: int (default 5432, must be between 1-65535)
- `database`: str (required)
- `username`: str (required)
- `password`: Optional[str] (for security, make it optional)
- `ssl_enabled`: bool (default True)
- `timeout`: int (default 30, must be positive)

In [ ]:
# TODO: Implement DatabaseConfig
class DatabaseConfig(BaseModel):
    pass

# Test your implementation
# db = DatabaseConfig(
#     host="postgres.example.com",
#     database="analytics",
#     username="analyst"
# )
# print(db.model_dump_json(indent=2))

# Test validation - this should fail (port too high)
# try:
#     bad_db = DatabaseConfig(
#         host="localhost",
#         port=99999,
#         database="test",
#         username="user"
#     )
# except Exception as e:
#     print(f"✅ Validation caught error: {e}")

## Exercise 2: Production Environment Validation

**Goal**: Extend `DatabaseConfig` to prevent localhost in production.

**Requirements**:
- Add `environment`: Literal["dev", "staging", "prod"]
- Add a `@model_validator` that ensures:
  - If `environment == "prod"`, `host` cannot be "localhost" or "127.0.0.1"
  - If `environment == "prod"`, `ssl_enabled` must be True

In [ ]:
# TODO: Implement ProductionDatabaseConfig
class ProductionDatabaseConfig(BaseModel):
    pass

# Test - this should work
# dev_db = ProductionDatabaseConfig(
#     host="localhost",
#     database="test",
#     username="dev",
#     environment="dev"
# )
# print("✅ Dev with localhost OK")

# Test - this should fail (localhost in prod)
# try:
#     prod_db = ProductionDatabaseConfig(
#         host="localhost",
#         database="prod_db",
#         username="prod_user",
#         environment="prod"
#     )
# except ValueError as e:
#     print(f"✅ Caught prod localhost error: {e}")

# Test - this should fail (SSL disabled in prod)
# try:
#     prod_db = ProductionDatabaseConfig(
#         host="prod.example.com",
#         database="prod_db",
#         username="prod_user",
#         environment="prod",
#         ssl_enabled=False
#     )
# except ValueError as e:
#     print(f"✅ Caught prod SSL error: {e}")

## Exercise 3: SQL Transformation Config

**Goal**: Create a model for SQL transformations.

**Requirements**:
- `name`: str (must be valid Python identifier)
- `sql`: str (required, cannot be empty or whitespace)
- `description`: Optional[str]
- `parameters`: Dict[str, Any] (default empty dict)
- `enabled`: bool (default True)

**Validators**:
- Validate `name` is a valid Python identifier (use `str.isidentifier()`)
- Validate `sql` is not empty or just whitespace (use `str.strip()`)

In [ ]:
# TODO: Implement TransformationConfig
class TransformationConfig(BaseModel):
    pass

# Test your implementation
# transform = TransformationConfig(
#     name="clean_sales",
#     sql="SELECT * FROM sales WHERE amount > 0",
#     description="Remove negative amounts",
#     parameters={"min_amount": 0}
# )
# print(transform.model_dump_json(indent=2))

# Test - invalid name (has hyphen)
# try:
#     bad_transform = TransformationConfig(
#         name="clean-sales",
#         sql="SELECT 1"
#     )
# except ValueError as e:
#     print(f"✅ Invalid name caught: {e}")

# Test - empty SQL
# try:
#     bad_transform = TransformationConfig(
#         name="test",
#         sql="   "  # Just whitespace
#     )
# except ValueError as e:
#     print(f"✅ Empty SQL caught: {e}")

## Exercise 4: File Format Config

**Goal**: Create configs for different file formats with format-specific options.

**Requirements**:
- Create `FileFormat` enum: CSV, PARQUET, JSON, AVRO
- Create `CompressionType` enum: NONE, GZIP, SNAPPY, LZ4
- Create `FileConfig` with:
  - `path`: str (required)
  - `format`: FileFormat (required)
  - `compression`: CompressionType (default NONE)
  - `options`: Dict[str, Any] (default empty)
  - Add validator: if format is CSV, options can have "delimiter" and "header"
  - Add validator: PARQUET and AVRO cannot use GZIP (not supported)

In [ ]:
# TODO: Implement FileFormat, CompressionType, and FileConfig

class FileFormat(str, Enum):
    pass

class CompressionType(str, Enum):
    pass

class FileConfig(BaseModel):
    pass

# Test CSV with options
# csv_file = FileConfig(
#     path="/data/sales.csv",
#     format=FileFormat.CSV,
#     options={"delimiter": "|", "header": True}
# )
# print(csv_file)

# Test Parquet with compression
# parquet_file = FileConfig(
#     path="/data/sales.parquet",
#     format=FileFormat.PARQUET,
#     compression=CompressionType.SNAPPY
# )
# print(parquet_file)

# Test invalid combination (Parquet + GZIP)
# try:
#     bad_file = FileConfig(
#         path="/data/test.parquet",
#         format=FileFormat.PARQUET,
#         compression=CompressionType.GZIP
#     )
# except ValueError as e:
#     print(f"✅ Invalid compression caught: {e}")

## Exercise 5: Data Quality Rules

**Goal**: Create a model for data quality validation rules.

**Requirements**:
- `rule_name`: str (valid identifier)
- `column`: str (required)
- `rule_type`: Literal["not_null", "unique", "range", "regex", "custom"]
- `parameters`: Dict[str, Any] (default empty)
- `severity`: Literal["warning", "error"] (default "error")

**Validators**:
- If `rule_type == "range"`, parameters must have "min" or "max"
- If `rule_type == "regex"`, parameters must have "pattern"
- If `rule_type == "custom"`, parameters must have "function"

In [ ]:
# TODO: Implement DataQualityRule
class DataQualityRule(BaseModel):
    pass

# Test not_null rule
# rule1 = DataQualityRule(
#     rule_name="check_customer_id",
#     column="customer_id",
#     rule_type="not_null"
# )
# print(rule1)

# Test range rule
# rule2 = DataQualityRule(
#     rule_name="check_age_range",
#     column="age",
#     rule_type="range",
#     parameters={"min": 0, "max": 120}
# )
# print(rule2)

# Test invalid range rule (missing parameters)
# try:
#     bad_rule = DataQualityRule(
#         rule_name="bad_range",
#         column="value",
#         rule_type="range"
#         # Missing min/max!
#     )
# except ValueError as e:
#     print(f"✅ Missing range parameters caught: {e}")

## Exercise 6: Complete ETL Pipeline Config

**Goal**: Combine all previous models into a complete ETL configuration.

**Requirements**:
- `name`: str (required)
- `source`: FileConfig (required)
- `transformations`: List[TransformationConfig] (at least one required)
- `quality_rules`: List[DataQualityRule] (default empty)
- `destination`: DatabaseConfig (required)
- `schedule`: Optional[str] (cron expression)
- `enabled`: bool (default True)

**Validators**:
- Ensure `transformations` list has at least one item
- Ensure all transformation names are unique
- Ensure all quality rule names are unique

In [ ]:
# TODO: Implement ETLPipelineConfig
class ETLPipelineConfig(BaseModel):
    pass

# Create a complete ETL pipeline
# etl = ETLPipelineConfig(
#     name="daily_sales_etl",
#     source=FileConfig(
#         path="/data/sales.csv",
#         format=FileFormat.CSV,
#         options={"delimiter": ",", "header": True}
#     ),
#     transformations=[
#         TransformationConfig(
#             name="filter_valid",
#             sql="SELECT * FROM source WHERE amount > 0"
#         ),
#         TransformationConfig(
#             name="add_timestamp",
#             sql="SELECT *, CURRENT_TIMESTAMP as processed_at FROM filtered"
#         )
#     ],
#     quality_rules=[
#         DataQualityRule(
#             rule_name="check_amount",
#             column="amount",
#             rule_type="range",
#             parameters={"min": 0}
#         )
#     ],
#     destination=DatabaseConfig(
#         host="warehouse.example.com",
#         database="analytics",
#         username="etl_user"
#     ),
#     schedule="0 2 * * *"  # Daily at 2 AM
# )
# print(etl.model_dump_json(indent=2))

## Exercise 7: Advanced - Union Types

**Goal**: Handle multiple source types in one pipeline.

**Requirements**:
- Create `APIConfig` model:
  - `url`: str (required)
  - `method`: Literal["GET", "POST"] (default "GET")
  - `headers`: Dict[str, str] (default empty)
  - `timeout`: int (default 30, must be positive)
- Create `Source` type alias as Union[FileConfig, DatabaseConfig, APIConfig]
- Modify ETLPipelineConfig to accept `source: Source`

In [ ]:
# TODO: Implement APIConfig and Source union
class APIConfig(BaseModel):
    pass

# Source = Union[FileConfig, DatabaseConfig, APIConfig]

# class FlexibleETLConfig(BaseModel):
#     name: str
#     source: Source  # Now accepts any source type!
#     # ... rest of fields

# Test with different source types
# file_etl = FlexibleETLConfig(
#     name="file_pipeline",
#     source=FileConfig(path="/data/file.csv", format=FileFormat.CSV)
# )

# api_etl = FlexibleETLConfig(
#     name="api_pipeline",
#     source=APIConfig(url="https://api.example.com/data")
# )

# db_etl = FlexibleETLConfig(
#     name="db_pipeline",
#     source=DatabaseConfig(host="db.example.com", database="src", username="reader")
# )

## Bonus Exercise: Schema Migration

**Challenge**: Create a model for database schema migrations.

**Requirements**:
- `version`: str (format: "vX.Y.Z" where X, Y, Z are integers)
- `description`: str (required)
- `up_sql`: str (SQL to apply migration)
- `down_sql`: str (SQL to rollback migration)
- `applied_at`: Optional[str] (ISO timestamp)
- `checksum`: Optional[str] (MD5 hash of up_sql)

**Validators**:
- Validate version format with regex
- Ensure up_sql and down_sql are not empty
- Auto-compute checksum from up_sql if not provided

In [ ]:
# TODO: Implement SchemaMigration
# Hint: Use field_validator and model_validator
# Hint: For checksum, use hashlib.md5

import hashlib
import re

class SchemaMigration(BaseModel):
    pass

# migration = SchemaMigration(
#     version="v1.0.0",
#     description="Add user_id column",
#     up_sql="ALTER TABLE orders ADD COLUMN user_id INTEGER",
#     down_sql="ALTER TABLE orders DROP COLUMN user_id"
# )
# print(migration.model_dump_json(indent=2))

---

## 🎉 Completion Checklist

- [ ] Exercise 1: DatabaseConfig with constraints
- [ ] Exercise 2: Production environment validation
- [ ] Exercise 3: SQL transformation config
- [ ] Exercise 4: File format config with validators
- [ ] Exercise 5: Data quality rules
- [ ] Exercise 6: Complete ETL pipeline config
- [ ] Exercise 7: Union types for multiple sources
- [ ] Bonus: Schema migration model

Once complete, check your solutions against `solutions.ipynb`!